In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import tensorflow as tf
tfk = tf.keras
tfkl = tf.keras.layers
import sklearn as skl
import scipy
import random


import warnings
import os

seed = 69
tf.random.set_seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)
tf.compat.v1.set_random_seed(seed)

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

In [8]:
df = pd.read_excel('FinancialMarketData.xlsx', 'EWS')